In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---

# Human

## Zheng

In [ ]:
zheng = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Zheng/data.h5")
utils.peek(zheng, "build/pbmc/Zheng")
zheng.obs.head()

In [ ]:
zheng.obs.dtypes

In [ ]:
opt_model_kwargs = dict(h_dim=512)
zheng_model = cb.directi.fit_DIRECTi(
    zheng, zheng.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
zheng.latent = zheng_model.inference(zheng)

In [ ]:
ax = zheng.visualize_latent("cell_ontology_class", size=0.1, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/pbmc/Zheng/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = zheng.visualize_latent("cell_type1", size=0.1, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/pbmc/Zheng/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
zheng.write_dataset("build/pbmc/Zheng/Zheng.h5")

In [ ]:
%%capture capio
zheng_models = [zheng_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    zheng_models.append(cb.directi.fit_DIRECTi(
        zheng, zheng.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
zheng_blast = cb.blast.BLAST(zheng_models, zheng)
zheng_blast.save("build/pbmc/Zheng")

In [ ]:
with open("build/pbmc/Zheng/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/pbmc/Zheng/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(zheng_blast, "build/pbmc/Zheng")

In [ ]:
%%writefile build/pbmc/Zheng/predictable.txt
cell_ontology_class
cell_type1